# Ideas

- consider event only if there is a significant (score > 0.2) and same type event is near (2-3 hour) it 
- power averaging

- try with 4 folds
- try with 6 folds
- try with full data

- try again gaussian loss

What works and gives boost but overfits:
- LSTM encoder (0.772/~0.722)
- resnet50 (0.780/0.722)
- duration 17280 (0.772/0.722)

In [1]:
import json
import pandas as pd
from pathlib import Path
from hydra import initialize, compose
from hydra.core.global_hydra import GlobalHydra
from src.utils.metrics import event_detection_ap

from src.utils.post_process import post_process_for_seg
import jupyter_black
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import optuna


jupyter_black.load()
plt.style.use("ggplot")

INFERENCE = False
EXP_NAME = "transformer_dur_8640_folds"
RUN_NAME = "run0"
TYPE = "score"

if INFERENCE:
    RESULT_DIR = Path("../output/inference") / EXP_NAME  # / RUN_NAME
    hydra_result_dir = Path("../output/train") / "transformer_dur_8640_folds" / RUN_NAME
else:
    RESULT_DIR = Path("../output/train") / EXP_NAME / RUN_NAME
    hydra_result_dir = Path("../output/train") / EXP_NAME / RUN_NAME


def load_config(result_dir: Path):
    # clear previous initialization
    GlobalHydra.instance().clear()

    # initialize hydra
    config_path = result_dir / ".hydra"
    initialize(config_path=config_path.as_posix())
    # load the config
    cfg = compose(config_name="config")

    return cfg


cfg = load_config(hydra_result_dir)

if INFERENCE:
    preds = np.load(RESULT_DIR / f"preds.npy")
    keys = np.load(RESULT_DIR / f"keys.npy")
else:
    preds = np.load(RESULT_DIR / f"preds_{TYPE}.npy")
    labels = np.load(RESULT_DIR / f"labels_{TYPE}.npy")
    keys = np.load(RESULT_DIR / "keys.npy")

gt_df = pd.read_csv(Path(cfg.dir.data_dir) / "train_events.csv")
gt_df = gt_df[gt_df["series_id"].isin(cfg.split.valid_series_ids)].dropna().reset_index(drop=True)

/tmp/ipykernel_1662410/251296522.py:39: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path=config_path.as_posix())


# Optimize postprocess parameters using Optuna

In [2]:
with open(Path(cfg.dir.processed_dir) / "train" / "series_lens.json") as f:
    series_lens = json.load(f)

pred_df = post_process_for_seg(keys, preds, series_lens, score_th=0.0001, distance=70, offset=8)
# pred_df = pred_df.to_pandas()

score = event_detection_ap(gt_df, pred_df)

score, len(pred_df)

(0.7672724347834871, 54999)

In [18]:
# fold 5_0 - 0.765 / 0.733 - rank 1
# fold 5_3 - 0.772 / 0.733 - rank 3

# fold 10_2 - 0.0001 / 85 / 0.811 / 0.729 (0.709 for the best loss) - rank 6
# fold 10_4 - 0.0001 / 70 / 0.789 / 0.734 - rank 2
# fold 10_6 - 0.0001 / 70 / 0.779 / 0.733 - rank 4
# fold 10_7 - 0.0001 / 80 / 0.792 / 0.729 - rank 5
# fold 10_9 - 0.0001 / 80 / 0.812 / 0.722

In [ ]:
def objective(trial: optuna.Trial):
    # score_th = 0.005
    score_th = trial.suggest_float("score_th", 0, 0.5)
    distance = trial.suggest_int("distance", 30, 400)

    pred_df: pl.DataFrame = post_process_for_seg(keys, preds, score_th=score_th, distance=distance)
    score = event_detection_ap(gt_df, pred_df)

    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print(study.best_params)

In [ ]:
def plot_random_sample(gt_df, keys, preds, labels, num_samples=1, num_chunks=10):
    # get series ids
    series_ids = np.array(list(map(lambda x: x.split("_")[0], keys)))
    unique_series_ids = np.unique(series_ids)

    # get random series
    random_series_ids = np.random.choice(unique_series_ids, num_samples)

    for i, random_series_id in enumerate(random_series_ids):
        # get random series
        series_idx = np.where(series_ids == random_series_id)[0]
        this_series_preds = preds[series_idx].reshape(-1, 3)
        this_series_labels = labels[series_idx].reshape(-1, 3)

        # split series
        this_series_preds = np.split(this_series_preds, num_chunks)
        this_series_labels = np.split(this_series_labels, num_chunks)
        this_series_len = [0] + [len(x) for x in this_series_labels]
        this_series_len = np.cumsum(this_series_len)

        gt_df = gt_df[gt_df["series_id"] == random_series_id]

        fig, axs = plt.subplots(num_chunks, 1, figsize=(20, 5 * num_chunks))

        if num_chunks == 1:
            axs = [axs]

        for j in range(num_chunks):
            this_series_preds_chunk = this_series_preds[j]
            this_series_labels_chunk = this_series_labels[j]

            # get onset and wakeup idx
            gt_tmp = gt_df[
                (gt_df["step"] >= this_series_len[j]) & (gt_df["step"] <= this_series_len[j + 1])
            ]
            onset_idx = gt_tmp.loc[gt_tmp["event"] == "onset", "step"].to_list()
            onset_idx = onset_idx - this_series_len[j]
            wakeup_idx = gt_tmp.loc[gt_tmp["event"] == "wakeup", "step"].to_list()
            wakeup_idx = wakeup_idx - this_series_len[j]

            axs[j].plot(this_series_preds_chunk[:, 0], label="pred_sleep")
            axs[j].plot(this_series_preds_chunk[:, 1], label="pred_onset")
            axs[j].plot(this_series_preds_chunk[:, 2], label="pred_wakeup")
            axs[j].vlines(onset_idx, 0, 1, label="onset", linestyles="dashed", color="C1")
            axs[j].vlines(wakeup_idx, 0, 1, label="wakeup", linestyles="dashed", color="C2")
            axs[j].set_ylim(0, 1)
            axs[j].set_title(f"series_id: {random_series_id} chunk_id: {j}")
            axs[j].legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0)

    plt.tight_layout()


plot_random_sample(gt_df, keys, preds, labels, num_chunks=10)

# Inference

In [5]:
# Config
BEST_MODEL = "ensemble"
FOLD = 0

DURATION = 8640
DOWNSAMPLE_RATE = 2
PHASE = "train"
EXP_NAME = "transformer_best_folds"

In [6]:
!python -m run.inference\
    dir=local\
    model.params.encoder_name=resnet34\
    model.params.encoder_weights=null\
    num_workers=12\
    exp_name=$EXP_NAME\
    weight.run_name=single\
    batch_size=8\
    duration=$DURATION\
    downsample_rate=$DOWNSAMPLE_RATE\
    pp.score_th=0.0015\
    pp.distance=70\
    phase=$PHASE\
    best_model=$BEST_MODEL

Global seed set to 42
[5.0GB(+3.9GB):2.0sec] load test dataloader 
List of models: ['/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_0.pth', '/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_2.pth', '/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_4.pth', '/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_6.pth', '/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_7.pth', '/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_3.pth']
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_0.pth
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_2.pth
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_4.pth
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_6.pth
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_7.pth
load weight from /home/alex/Ka